In [1]:
import numpy as np
from sklearn.metrics import accuracy_score
import torch
import torch.nn.functional as F
from ogb.nodeproppred import PygNodePropPredDataset, Evaluator
import torch_geometric.transforms as T
from torch_geometric.data import DataLoader
from torch_geometric.nn import SAGEConv

In [2]:
dataset = PygNodePropPredDataset(name='ogbn-arxiv', 
                                 root='../data/dataset/',
                                 transform=T.ToSparseTensor())

In [3]:
torch.cuda.is_available()

True

In [4]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)

In [5]:
data = dataset[0]
data.adj_t = data.adj_t.to_symmetric()

In [6]:
data

Data(adj_t=[169343, 169343, nnz=2315598], node_year=[169343, 1], x=[169343, 128], y=[169343, 1])

In [7]:
data = data.to(device)

In [8]:
split_idx = dataset.get_idx_split()
train_idx = split_idx['train'].to(device)

In [9]:
evaluator = Evaluator(name='ogbn-arxiv')

In [10]:
data['x'].shape[1]

128

In [11]:
data['y'][:, 0].unique().shape[0]

40

In [12]:
n_iters = 3000
epochs = 500
log_steps = 25
input_dim = data['x'].shape[1]
hidden_channels = input_dim * 2
output_dim = data['y'][:, 0].unique().shape[0]
lr_rate = 0.001

In [13]:
class ThreeLayer(torch.nn.Module):
    def __init__(self, input_dim, hidden_channels, output_dim):
        super(ThreeLayer, self).__init__()
        
        self.convs = torch.nn.ModuleList()
        self.bns = torch.nn.ModuleList()
        
        self.convs.append(SAGEConv(input_dim, hidden_channels))
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        
        self.convs.append(SAGEConv(hidden_channels, hidden_channels))
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        
        self.convs.append(SAGEConv(hidden_channels, output_dim))
        

    def forward(self, x, adj_t):
        x = self.convs[0](x, adj_t)
        x = self.bns[0](x)
        x = F.relu(x)
        
        x = self.convs[1](x, adj_t)
        x = self.bns[1](x)
        x = F.relu(x)
        
        x = self.convs[2](x, adj_t)
        return x.log_softmax(dim=-1)

In [14]:
model = ThreeLayer(input_dim, hidden_channels, output_dim).to(device)

In [15]:
def train(model, data, train_idx, optimizer):
    model.train()

    optimizer.zero_grad()
    out = model(data.x, data.adj_t)[train_idx]
    loss = F.nll_loss(out, data.y.squeeze(1)[train_idx])
    loss.backward()
    optimizer.step()

    return loss.item()

In [16]:
@torch.no_grad()
def test(model, data, split_idx, evaluator):
    model.eval()

    out = model(data.x, data.adj_t)
    y_pred = out.argmax(dim=-1, keepdim=True)

    train_acc = evaluator.eval({
        'y_true': data.y[split_idx['train']],
        'y_pred': y_pred[split_idx['train']],
    })['acc']
    valid_acc = evaluator.eval({
        'y_true': data.y[split_idx['valid']],
        'y_pred': y_pred[split_idx['valid']],
    })['acc']
    test_acc = evaluator.eval({
        'y_true': data.y[split_idx['test']],
        'y_pred': y_pred[split_idx['test']],
    })['acc']

    return train_acc, valid_acc, test_acc

In [17]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr_rate)
for epoch in range(1, 1 + epochs):
    loss = train(model, data, train_idx, optimizer)
    result = test(model, data, split_idx, evaluator)

    if epoch % log_steps == 0:
        train_acc, valid_acc, test_acc = result
        print(f'Epoch: {epoch:02d}, '
              f'Loss: {loss:.4f}, '
              f'Train: {100 * train_acc:.2f}%, '
              f'Valid: {100 * valid_acc:.2f}% '
              f'Test: {100 * test_acc:.2f}%')

Epoch: 25, Loss: 1.2518, Train: 53.40%, Valid: 57.69% Test: 59.84%
Epoch: 50, Loss: 1.0355, Train: 69.01%, Valid: 68.68% Test: 68.36%
Epoch: 75, Loss: 0.9302, Train: 72.02%, Valid: 70.50% Test: 69.64%
Epoch: 100, Loss: 0.8518, Train: 73.80%, Valid: 70.86% Test: 70.09%
Epoch: 125, Loss: 0.7784, Train: 75.96%, Valid: 71.14% Test: 69.99%
Epoch: 150, Loss: 0.7042, Train: 77.67%, Valid: 70.72% Test: 68.89%
Epoch: 175, Loss: 0.6343, Train: 79.27%, Valid: 70.36% Test: 68.34%
Epoch: 200, Loss: 0.5582, Train: 81.76%, Valid: 70.53% Test: 68.58%
Epoch: 225, Loss: 0.4937, Train: 83.61%, Valid: 70.38% Test: 68.37%
Epoch: 250, Loss: 0.4358, Train: 86.08%, Valid: 70.09% Test: 68.02%
Epoch: 275, Loss: 0.3835, Train: 88.06%, Valid: 69.59% Test: 67.25%
Epoch: 300, Loss: 0.3378, Train: 89.89%, Valid: 69.64% Test: 67.41%
Epoch: 325, Loss: 0.2986, Train: 91.33%, Valid: 68.97% Test: 66.80%
Epoch: 350, Loss: 0.2633, Train: 92.81%, Valid: 68.68% Test: 66.39%
Epoch: 375, Loss: 0.2318, Train: 94.16%, Valid: 68.